In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from os.path import join as pjoin

from matplotlib import font_manager
font_manager.fontManager.addfont('./supportfiles/arial.ttf')

# define default plots
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams.update({'font.size': 12, 'font.family': 'Arial', 'mathtext.fontset': 'stix'})

In [ ]:
def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])
    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value

### Frame Displacement calculation

In [ ]:
# define path
dataset_path = 'PATH TO DATASET'
fmripreppath = f'{dataset_path}/derivatives/fmriprep'
bq_path = 'PATH TO SAVE RESULTS' 
fig_path = 'PATH TO SAVE FIGURES' 

In [ ]:
# define motion variables
mot_params = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']

# define task name and metrics
taskname = 'floc' # 'coco', 'prf', 'imagenet'
metric = 'median'
# generate subject list
if taskname == 'imagenet': subs = ['sub-{:02d}'.format(i) for i in range(1,31)]
else: subs = ['sub-{:02d}'.format(i) for i in range(1,10)]

# initialize the data
all_fd = np.zeros((len(subs),), dtype=object)
all_fd_quota = np.zeros((len(subs),), dtype=object)
if metric =='mean': quota_metric = np.nanmean
elif metric == 'median' : quota_metric = np.nanmedian

# subject loop
for i, sub in enumerate(subs):
    # fetch sessions
    sessions = sorted([_ for _ in os.listdir(pjoin(fmripreppath, sub)) if taskname in _])
    # initialize
    run_all_fd, run_fd_quota = [], []
    # session loop
    for session in sessions:
        confound_files = [_ for _ in os.listdir(pjoin(fmripreppath, sub, session, 'func')) if 'timeseries.tsv' in _]
        for file in confound_files:
            df = pd.read_csv(pjoin(fmripreppath,f'{sub}/{session}/func/{file}'), sep='\t')
            # gather all  FD
            FmDp = np.sum(tuple([np.abs(df[_].values) for _ in mot_params]), axis=0)
            run_all_fd.extend(list(FmDp))
            # calc FD quota
            FD_index = quota_metric(FmDp)
            run_fd_quota.append(FD_index)
    all_fd[i] = np.array(run_all_fd)
    all_fd_quota[i] = np.array(run_fd_quota)

# save
np.save(pjoin(bq_path, f'sub-{taskname}_allfd.npy'), all_fd)
if metric == 'median': np.save(pjoin(bq_path, f'sub-{taskname}_allmedianfd.npy'), all_fd_quota)
if metric == 'mean': np.save(pjoin(bq_path, f'sub-{taskname}_allmeanfd.npy'), all_fd_quota)

### Frame Displacement load for checking

In [ ]:
all_fd_mean = np.load(pjoin(bq_path, 'sub_allmeanfd.npy'), allow_pickle=True)
all_fd_median = np.load(pjoin(bq_path, 'sub_allmedianfd.npy'), allow_pickle=True)
all_fd = np.load(pjoin(bq_path, 'sub_allfd.npy'), allow_pickle=True)

### Plots

#### for imagenet

In [ ]:
# plots settings
mpl.rcParams.update({'font.size': 10.5, 'font.family': 'Arial', 'mathtext.fontset': 'stix'})
fig,ax = plt.subplots(figsize=(7,1.2))
width=0.7
font_size = 10.5

# load data
taskname = 'imagenet'
all_fd_mean = np.load(pjoin(bq_path, f'sub-{taskname}_allmeanfd.npy'), allow_pickle=True)
all_fd_median = np.load(pjoin(bq_path, f'sub-{taskname}_allmedianfd.npy'), allow_pickle=True)
all_fd = np.load(pjoin(bq_path, f'sub-{taskname}_allfd.npy'), allow_pickle=True)

# index appointment
draw_index =  all_fd 
index_name =  'FD(mm)' 

# draw for each subject
for i, data in enumerate(draw_index) :
    parts = ax.violinplot(data, positions=[i], widths=width,showextrema=False,showmedians=False, vert=True)
    for pc in parts['bodies']:
        pc.set_facecolor('#81C6E8')
        pc.set_alpha(0.55)
    q1, medians, q3 = np.percentile(np.sort(data), [25, 50, 75], axis=0)
    whiskers = np.array([adjacent_values(np.sort(data), q1, q3)])
    whiskers_min, whiskers_max = whiskers[:, 0], whiskers[:, 1]
    ax.scatter(i, medians, marker='o', color='#B73E3E', s=10, zorder=3)
    ax.vlines(i, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1.2, alpha=0.3)
    ax.vlines(i, q1, q3, color='red',edgecolors='#E97777', linestyle='-', lw=3.6)
# reference line
ax.axhline(y=0.5, color='gray', ls='--')

# axis settings
# x-axis
ax.set_xlim([-0.9,29.52])
ax.set_xticks(np.arange(30))
ax.set_xticklabels(['{:d}'.format(i+1) for i in np.arange(30)])
ax.set_xlabel(f'subject',fontsize=font_size)
# y-axis
ax.set_ylim([0,1.25])
# ax.set_yticks([0,1,2,3])
# ax.set_yticklabels([],fontsize=10.5)
ax.set_ylabel(index_name, fontsize=font_size)
# spinses and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(2.0)
ax.spines['bottom'].set_linewidth(2.0)
ax.tick_params(labelsize=font_size, width=1, length=3, direction="in", pad=2)
# save out
fig.savefig(pjoin(fig_path, f'{index_name}-{taskname}.jpg'),dpi=72*4,bbox_inches='tight')
plt.show()

#### plot for coco / prf / floc

In [ ]:
# plots setting
mpl.rcParams.update({'font.size': 10.5, 'font.family': 'Arial', 'mathtext.fontset': 'stix'})
fig,axs = plt.subplots(1,3,figsize=(7.2,1))
width=0.7
font_size = 10.5

# load data
tasknames = ['coco', 'floc', 'prf']
for j, ax in enumerate(axs):
    # load data
    taskname = tasknames[j]
    all_fd_mean = np.load(pjoin(bq_path, f'sub-{taskname}_allmeanfd.npy'), allow_pickle=True)
    all_fd_median = np.load(pjoin(bq_path, f'sub-{taskname}_allmedianfd.npy'), allow_pickle=True)
    all_fd = np.load(pjoin(bq_path, f'sub-{taskname}_allfd.npy'), allow_pickle=True)
    # index appointment
    draw_index =all_fd #  all_fd_mean # all_fd_median # 
    index_name =  'FD(mm)' # 'mean FD(mm)'#'median FD(mm)' #
    # draw for each subject
    for i, data in enumerate(draw_index) :
        parts = ax.violinplot(data, positions=[i], widths=width,showextrema=False,showmedians=False, vert=True)
        for pc in parts['bodies']:
            pc.set_facecolor('#81C6E8')
            pc.set_alpha(0.55)
        q1, medians, q3 = np.percentile(np.sort(data), [25, 50, 75], axis=0)
        whiskers = np.array([adjacent_values(np.sort(data), q1, q3)])
        whiskers_min, whiskers_max = whiskers[:, 0], whiskers[:, 1]
        ax.scatter(i, medians, marker='o', color='#B73E3E', s=10, zorder=3)
        ax.vlines(i, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1.2, alpha=0.3)
        ax.vlines(i, q1, q3, color='red',edgecolors='#E97777', linestyle='-', lw=3.6)
    # reference line
    ax.axhline(y=0.5, color='gray', ls='--')
    # axis settings
    # x-axis
    ax.set_xticks(np.arange(9))
    ax.set_xticklabels(['{:d}'.format(i+1) for i in np.arange(9)])
    ax.set_xlim([-0.8,8.5])
    # y-axis
    ax.set_ylim([0,1.25])
    ax.set_yticks([0,.5,1])
    # ax.set_yticklabels([],fontsize=10.5)
    if j == 0:
        ax.set_ylabel(index_name, fontsize=font_size)
    ax.set_xlabel(f'subject',fontsize=font_size)#({taskname})
    # spines and ticks
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(2.0)
    ax.spines['bottom'].set_linewidth(2.0)
    ax.tick_params(labelsize=font_size, width=1, length=3, direction="in", pad=2)
# save out
fig.savefig(fig_path, pjoin(f'{index_name}-{tasknames}.jpg'),dpi=72*4,bbox_inches='tight')
plt.show()